In [1]:
import apodeixi.xli as xli
import os
import datetime

In [2]:
EXCEL_FOLDER       = os.getcwd() + '/../'+ 'excel-postings'
EXPECTATIONS_FILE  = 'S1-breakdown-posting.xlsx'
EXPECTATIONS_RANGE = 'a4:e100'
SHEETS             = ['Ew' + str(idx) for idx in range(10)]
SHEETS

['Ew0', 'Ew1', 'Ew2', 'Ew3', 'Ew4', 'Ew5', 'Ew6', 'Ew7', 'Ew8', 'Ew9']

In [3]:
_PRODUCT              = 'product'
_JOURNEY              = 'journey'
_PLAN_TYPE            = 'planType'
_SCENARIO             = 'scenario'
_ENVIRONMENT          = 'environment'
_SCORING_CYCLE        = 'scoringCycle'
_SCORING_MATURITY     = 'scoringMaturity'
_ESTIMATED_BY         = 'estimatedBy'
_ESTIMATED_ON         = 'estimatedOn'
_RECORDED_BY          = 'recordedBy'
_PURSUIT              = 'pursuit'
_BREAKDOWN_TYPE       = 'breakdownType'

In [4]:
CTX                      = {}
CTX[_PURSUIT]            = 'S1-AccelerateCloud'
CTX[_SCENARIO]           = 'Baseline'
CTX[_ENVIRONMENT]        = 'Production'
CTX[_SCORING_CYCLE]      = 'FY22'
CTX[_SCORING_MATURITY]   = 'Draft'
#CTX[_ESTIMATED_BY]       = 'a'
CTX[_ESTIMATED_ON]       =  datetime.datetime.strptime('2021-04-27', '%Y-%m-%d')
CTX[_RECORDED_BY]        = 'alejandro.hernandez@finastra.com'
CTX[_BREAKDOWN_TYPE]     = 'Amplify initiative breakdown'

In [5]:
MANIFESTS_REPO  = os.getcwd() + '/../'+ 'manifests-repo'
L0_SHEET       = "Generated"
l0_url         = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + L0_SHEET
l0_range       = "A1:G30"

In [6]:
store        = xli.UID_Store(start_at_0=True)
L1_ids       = []
LINKS        = []
for idx in range(len(SHEETS)):
    uid      = store.generateUID(acronym='W', parent_UID=None)
    L1_ids.append(uid)
    LINKS.append(xli.L1L2_Link(L1_UID         = 'W.0' + str(idx), \
                           L2_URL         = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + SHEETS[idx], \
                           L2_excel_range = EXPECTATIONS_RANGE))
L1_ids

['W.00',
 'W.01',
 'W.02',
 'W.03',
 'W.04',
 'W.05',
 'W.06',
 'W.07',
 'W.08',
 'W.09']

In [7]:
builder = xli.BreakdownBuilder(ctx=CTX, links=LINKS, l0_url=l0_url, l0_excel_range=l0_range, \
                           manifests_repo_dir=MANIFESTS_REPO, uid_store=store)
builder.build()

apiVersion: breakdown.a6i.io/v1dev
breakdown:
  defined_by: alejandro.hernandez@finastra.com
  defined_on: 2021-04-27 00:00:00
  streams:
    W.00: Cloud value proposition
    W.00-detail:
      Description: Define value proposition for customers on the Cloud
      Lead: "Product Strategy \u2013 Mike S"
      P metric: TBD
      R metric: TBD
      Support: Andrei
      expectations:
        E.00.00: Analyze segmentation
        E.00.00-detail:
          acceptance-criteria-artifacts:
            A.00.00.00: 'Segmentation tree: consolidated tree of segments (verticals,
              geos, tiers, and breakouts for either) '
            A.00.00.01: Size at each leaf in the segmentation tree
            A.00.00.02: Quantification of how much of that could be cloud
          description: Break Finastra's space into segments and for each measure the
            appetite for cloud
        E.00.01: Analyze "Jobs to be done" / Personas
        E.00.01-detail:
          acceptance-criteria-arti

            A.02.02.03: Architecture blueprint for Monitoring Platform
          description: Define the scope of FusionOperate capabilities in a way that
            it addresses all cross-cutting concerns across Finastra products going
            to cloud, so that they are addressed once, across the whole DevSecOps
            spectrum (CI/CD, reliability model, security model, etc.)
        E.02.03: Deliver FusionOperate
        E.02.03-detail:
          acceptance-criteria-artifacts:
            A.02.03.00: 'Developer productivity KPIs set and met '
            A.02.03.01: Customer onboarding KPIs set and met. KPIs targets must relate
              to the modernization stream's business outcome KPIs.
            A.02.03.02: Cost-of-operation KPIs set and met. KPIs targets must relate
              to the modernization stream's business outcome KPIs.
            A.02.03.03: Speed-of-upgrades KPIs set and met. KPIs targets must relate
              to the modernization stream's busi

    W.08: Customer Support for native Cloud solutions
    W.08-detail:
      Description: Implement Cloud customer support model across DevOps, IT and customer
        support
      Lead: 'Michael '
      P metric: Delivery of milestones per product
      R metric: Number of products with customer self-service capabilities supportability
        embedded made available to customers
      Support: Leigh
      expectations:
        E.08.00: Align with Fusion Operate
        E.08.00-detail:
          acceptance-criteria-artifacts:
            A.08.00.00: .nan
          description: Define how customer support would work for each of the service
            operating models that rely on Fusion Operate (FusionCloud, Cloud@Scale,
            FMS, and partner-hosted)
    W.09: Talent transformation
    W.09-detail:
      Description: 'Define and deliver skills & capabilities required fordelivering
        Cloud faster across organization: Product, GCO, CCO.'
      Lead: Clare
      P metric: "

<h1>Scrap</h1>

In [40]:
'''
import pandas            as pd
import yaml              as _yaml
import os                as os
import math              as math
import apodeixi.xli      as xli
import datetime          as datetime
import sys               as _sys
import re                as _re
'''

In [108]:
class UID_Store:
    '''
    @param start_at_0 Boolean. If True, UIDs start at 0 (i.e., for acronym X the first UID is X.00). False
                      by default, so UIDs start at 1 (e.g., X.01)
    '''
    def __init__(self, start_at_0=False):
        self.start_at_0   = True
        self.UID_dict     = {}
        return
    
    def _parseParent(self, parentUID):
        if parentUID == None:
            # We are starting a new branch from the root
            parent_acronym         = None
            parent_nb              = ''
        else:
            REGEX                  = r"^([a-zA-Z]+).(([0-9]{2})(.[0-9]{2})*)$" # Match eg P.03 or SD.04.12 or C.03.10.11
            m                      = _re.match(REGEX, parentUID) 
            if m==None or len(m.groups()) != 4:
                raise ValueError('Badly given parentUID. Should be something like "P.03" or "SD.02.11". Instead it was "' 
                                    + parentUID + '"')
            parent_acronym         = m.group(1)
            parent_nb              = m.group(2)   
            
        return parent_acronym, parent_nb
    
    def _add(self, acronym, nb):
        '''
        Adds a new UID to the internal store and returns it
        '''
        if acronym not in self.UID_dict.keys():
            self.UID_dict[acronym] = [nb] 
        else:
            self.UID_dict[acronym].append(nb)
            
        return acronym + '.' + nb
            
    def _already_used(self, acronym, parent_nb):
        '''
        Returns a list of the existing entries in the store that begin with acronym + parent_nb.
        These entries are the numerical part of the UID, i.e., strings consisting only of digits
        If there are none, it returns an empty list
        '''
        if acronym not in self.UID_dict.keys():
            return []
        # By this time we can safely assume that UID_dict has acronym as a key
        else:
            return [x for x in self.UID_dict[acronym] if x.startswith(parent_nb)]
        
    def _next_code(self, number_codes):
        '''
        For a list of strings representing N-digit number codes only differing in the last 2 digits, 
        it returns a new 2-digit number code that is 1 larger than the largest number code in the input. 
        As a side effect, it might sort the 'number_codes'
        
        Since only 99 2-digit codes are supported, will raise an exception if the largest number code in the 
        input is 99
        '''
        if len(number_codes)==0:
            if self.start_at_0:
                return '00'
            else:
                return '01'
        number_codes.sort()
        last_code     = number_codes[len(number_codes)-1]
        if len(last_code) < 2:
            raise ValueError('Require all number codes to have at least 2 digits. Instead was given this:\n'
                             + 'number_codes=' + number_codes)
        a             = int(last_code[len(last_code)-2]) 
        b             = int(last_code[len(last_code)-1])
        if a==9 and b==9:
            raise ValueException('UID generation of children UIDs only support 2-digit extensions of a parent UID, '
                                + 'ranging from 01 to 99, so there is hard limit of 99 children and that has been exceeded.')
        if b<9:
            b_new = b+1
            a_new = a
        else: # We have a carry
            b_new = 0
            a_new = a+1
        new_code    = str(a_new) + str(b_new)
        return new_code

    def generateUID(self, acronym, parent_UID):

        parent_acronym, parent_nb    = self._parseParent(parent_UID)
                    
        already_used_codes           = self._already_used(acronym, parent_nb)
                    
        next_suffix                  = self._next_code(already_used_codes)
        
        if parent_nb == None or len(parent_nb)==0:
            new_nb = next_suffix
        else:
            new_nb = parent_nb + '.' + next_suffix
        
        return self._add(acronym, new_nb)

<h1>Processing</h1>

In [78]:
os.getcwd()

'C:\\Users\\aleja\\Documents\\Code\\chateauclaudia-labs\\apodeixi\\experiments'

In [79]:
##EXCEL_FOLDER       = os.getcwd() + '/../'+ 'excel-api'
EXCEL_FOLDER       = os.getcwd() + '/../'+ 'experiments'
EXPECTATIONS_FILE  = 'Amplify_S1_Expectations.xlsx'
EXPECTATIONS_RANGE = 'a4:e100'
SHEETS             = ['Ew' + str(idx) for idx in range(10)]
SHEETS

['Ew0', 'Ew1', 'Ew2', 'Ew3', 'Ew4', 'Ew5', 'Ew6', 'Ew7', 'Ew8', 'Ew9']

In [80]:
_PRODUCT              = 'product'
_JOURNEY              = 'journey'
_PLAN_TYPE            = 'planType'
_SCENARIO             = 'scenario'
_ENVIRONMENT          = 'environment'
_SCORING_CYCLE        = 'scoringCycle'
_SCORING_MATURITY     = 'scoringMaturity'
_ESTIMATED_BY         = 'estimatedBy'
_ESTIMATED_ON         = 'estimatedOn'
_RECORDED_BY          = 'recordedBy'
_PURSUIT              = 'pursuit'
_BREAKDOWN_TYPE       = 'breakdownType'

In [81]:
CTX                      = {}
CTX[_PURSUIT]            = 'S1-AccelerateCloud'
CTX[_SCENARIO]           = 'Baseline'
CTX[_ENVIRONMENT]        = 'Production'
CTX[_SCORING_CYCLE]      = 'FY22'
CTX[_SCORING_MATURITY]   = 'Draft'
#CTX[_ESTIMATED_BY]       = 'a'
CTX[_ESTIMATED_ON]       =  datetime.datetime.strptime('2021-04-27', '%Y-%m-%d')
CTX[_RECORDED_BY]        = 'alejandro.hernandez@finastra.com'
CTX[_BREAKDOWN_TYPE]     = 'Amplify initiative breakdown'

In [82]:
next_sheet  = SHEETS[0]
r           = xli.ExcelTableReader(url        = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + next_sheet,
                                  excel_range = EXPECTATIONS_RANGE)
df          = r.read()

In [83]:
df

,UID,Expectation,Description,Acceptance Criteria Artifact,Evidence of correctness
0,NaN,Analyze segmentation,Break Finastra's space into segments and for e...,Segmentation tree: consolidated tree of segmen...,"Data sources (provider, numerical and label da..."
1,NaN,NaN,NaN,Size at each leaf in the segmentation tree,NaN
2,NaN,NaN,NaN,Quantification of how much of that could be cloud,NaN
3,NaN,"Analyze ""Jobs to be done"" / Personas",Using Design Thinking and Clayton Christensens...,Narrated diagrams indicating the timeline-base...,Data sources (customer or 3rd party interviews)
4,NaN,Define Value propositions,Define logical bundle of Finastra capabilities...,"Vision Statement for Value Proposition, using ...",NaN
5,NaN,NaN,NaN,Logical diagram of capabilities,NaN
6,NaN,NaN,NaN,Mapping to the Finastra products that would pr...,NaN
7,NaN,NaN,NaN,"Mapping to the ""Jobs to be done"" diagrams with...",NaN
8,NaN,Measure Opportunities's size,"For each of the value propositions, establish ...",Revenue projection per proposition expressed a...,NaN
9,NaN,Decide areas to prioritize,Determine the priorities to which investment s...,A decision-making model to choose the proposit...,NaN


In [84]:
df.columns

Index(['UID', 'Expectation', 'Description', 'Acceptance Criteria Artifact',
       'Evidence of correctness'],
      dtype='object')

In [85]:
class L1L2_Link:
    def __init__(self, L1_UID, L2_URL, L2_excel_range):
        self.L1_UID             = L1_UID
        self.L2_URL             = L2_URL
        self.L2_excel_range     = L2_excel_range
        return

In [289]:
class BreakdownBuilder:
    '''
    Creates a Breakdown domain resource by aggregating data from multiple Excel sheets within the same 
    Excel spreadsheet.
    A Breakdown is basically a tree.
    One sheet contains the top-level breakdown, for the root of the tree (level 0) and its immediate children (level 1).
    For each children in level 1, there is a sheet that contains a breakdown into level 2 and level 3 for a 
    particular "relationship type" between level 1 and level 2. Different relationship types would be on different sheets.
    
    @ param uid_store A UID_Store, used to generate unique identifiers for each node in the Breakdown tree.
    '''
    def __init__(self, ctx, links, l0_url, l0_excel_range, manifests_repo_dir, uid_store):
        self.ctx                  = ctx
        self.links                = links
        self.l0_url               = l0_url
        self.l0_excel_range       = l0_excel_range
        self.manifests_repo_dir   = manifests_repo_dir
        self.uid_store            = uid_store
        return
    
    def build(self):
        user                            = self.ctx[_RECORDED_BY]
        pursuit                         = self.ctx[_PURSUIT]
        scoring_cycle                   = self.ctx[_SCORING_CYCLE]
        plan_maturity                   = self.ctx[_SCORING_MATURITY]
        environment                     = self.ctx[_ENVIRONMENT]
        planning_scenario               = self.ctx[_SCENARIO]
        breakdown_type                  = self.ctx[_BREAKDOWN_TYPE]
        
        estimation_date                 = self.ctx[_ESTIMATED_ON]
        
        manifest_dict                   = {}

        metadata                        = {'namespace': environment + '.' + scoring_cycle,
                                           'name':      pursuit     + '.' + planning_scenario, 
                                           'labels': {'scoringCycle': scoring_cycle,
                                                      'scenario': planning_scenario,
                                                      'pursuit': pursuit}}

        manifest_dict['apiVersion']     = 'breakdown.a6i.io/v1dev'
        manifest_dict['kind']           = 'Breakdown'
        manifest_dict['metadata']       = metadata
        
        # Plan maturity can be one of: 'Not done', 'Drafted', 'Checked', 'Published'
        manifest_dict['planMaturity']   = plan_maturity
        
        l0_dict                         = self._buildLevel1Breakdown()
        
        manifest_dict['breakdown']      = {'type': breakdown_type,  #'estimated_by': estimating_mgr,
                                           'defined_on': estimation_date,
                                           'defined_by': user,
                                          'streams': l0_dict}    
        
        
        streams_dict                    = manifest_dict['breakdown']['streams'] 
        
        for link in self.links:
            l2_key, l2_children                      = self._buildLevel2Breakdown(link) 
            '''
            link_dict                                = {}
            link_dict[l2_key]                        = l2_children
            streams_dict[link.L1_UID]  = link_dict
            '''
            l1_key                      = link.L1_UID + '-detail' # For example, 'W.03-detail'
            if l1_key not in streams_dict.keys():
                streams_dict[l1_key]    = {}
            streams_dict[l1_key][l2_key] = l2_children
            
        _yaml.dump(manifest_dict, _sys.stdout)

        with open(self.manifests_repo_dir + '/' + pursuit + '-breakdown.yaml', 'w') as file:
            _yaml.dump(manifest_dict, file)
    
    def _nice(txt):
        return txt.strip().lower().replace(' ', '-')
    
    def _is_blank(txt):
        '''
        Returns True if 'txt' is NaN or just spaces
        '''
        if type(txt)==float and math.isnan(txt):
            return True
        elif type(txt)==str:
            stripped_txt = BreakdownBuilder._strip(txt)
            return len(stripped_txt)==0
        else:
            return False
        
    def _strip(txt):
        '''
        Removes any whitespace or other "noise" from txt and return sit
        '''
        if type(txt)==float and math.isnan(txt):
            return ''
        stripped_txt = str(txt).replace('\n', '').strip(' ')
        return stripped_txt
    
    def _cleanup_UID(txt):
        '''
        If the 'txt' finishes in a number but lacks a dot required for UIDs (e.g., txt="W4" or "W-4" instead of "W.04")
        it will return the correctly formatted txt ("W.04")
        '''
        REGEX = '^[a-zA-Z -\._]*([0-9]+)$'
        m     = _re.match(REGEX, txt.strip(' \n'))
        if m==None or len(m.groups()) != 1:
            raise ValueError("Unable to parse '" + txt + "': expected all digits to be in a contiguous suffix.")
        nb    = int(m.group(1))
        if nb > 100:
            raise ValueError("Invalid UID '" + txt + "':' the numerical value " + str(nb) + " should not exceed 99")
        prefix = txt.split(str(nb))[0]
        prefix = prefix.strip(' \n-_.')
        if nb < 10:
            nb_txt = '0' + str(nb) # From '00' to '09'
        else:
            nb_txt = str(nb) # From '10' to '99'
        return prefix + "." + nb_txt

    def _buildLevel1Breakdown(self):
        reader                           = xli.ExcelTableReader(url=self.l0_url, excel_range=self.l0_excel_range)
        l0_df                            = reader.read()
        columns                          = l0_df.columns

        if len(columns) < 2:
            raise ValueException("Too few columns: need at least two columns, the first two of which should the "\
                                + "UID and the Title for a Level 1 breakout. Happened for url=" + url)
        UID                              = columns[0]
        TITLE                            = columns[1]        
        l0_dict                          = {}
        for row in l0_df.iterrows():
            l1_data                      = row[1]
            l1_uid                       = _cleanup_UID(l1_data[UID])
            l0_dict[l1_uid]              = l1_data[TITLE]
            l0_dict[l1_uid + '-detail']  = {}
            for idx in range(2, len(columns)):
                col                      = columns[idx]
                l1_details_dict          = l0_dict[l1_uid + '-detail']
                l1_details_dict[col]     = l1_data[col]

        return l0_dict
    
    def _buildLevel2Breakdown(self, link):
        '''
        Returns a pair of a key and a sub-dictionary for a portion of the YAML to be built by this class.
        
        Below is an example of the YAML fragment that callers can build from the sub-dictionary created in this function.
        The example correspond to an Excel spreadsheet block that has columns called: 
        'UID', 'Expectation', 'Description', 'Acceptance Criteria Artifact', 'Evidence of correctness'

        expectations:
          E.01.01: Segment market using "jobs-to-be-done"
          E.01.01-detail:
            description: Follow methodology from Clayton Christensen
            acceptance-criteria-artifacts:
            - AC.01.01.01: Analysis of verticals, tiers, and geos evidence-of-correctness: a written analysis
            - AC.01.01.02 Sizing per segment
              evidence-of-correctness: a chart with sizings
          E.01.02: Deliver on time
          E.01.02-detail:
            description: stick to agreed milestones
            acceptance-criteria-artifacts:
            - AC.01.02.01 Project plan showing dates and staffing levels
              evidence-of-correctness: a Gant chart
            - AC.01.02.02 Completion of GA version 
              evidence-of-correctness: a monitoring report proving delivery to production
        '''    
        url                 = link.L2_URL
        excel_range         = link.L2_excel_range
        reader              = xli.ExcelTableReader(url, excel_range)

        l2l3_df             = reader.read()


        if len(l2l3_df.columns) != 5:
            raise ValueError ("Badly formatted breakdown: should have exactly five columns, corresponding to: " \
                              + "'UID', name of the L1-L2 relationship, descriptive attribute of L2, "\
                              + "name of the L2-L3 relationship, descriptive attribute of L3." \
                              + "  Error whiln processing range= '" + excel_range + "' and url=\n\t" + url)
        
        # Column names in CAPs
        columns             = l2l3_df.columns
        L2                  = columns[1]
        L2_DESC             = columns[2]
        L3                  = columns[3]
        L3_DESC             = columns[4]
        
        # Data that is different for each ith cycle of loop
        l2_dict_i           = {}
        l3_UID_i            = None
        
        # Data that changes on the nth time a new Level 2 is entered
        l3_children_n       = {}
        l2_UID_n            = None

        # Data constructed in subsequent cycles of loop
        all_l2s_dict        = {}
        
        # YAML-friendly keys derived from the columns (e.g., "Acceptance Criteria" becomes "acceptance-criteria")
        Y_L2                = BreakdownBuilder._nice(L2) + "s" # Plural, as we have multiple level 2 things
        Y_L2_DESC           = BreakdownBuilder._nice(L2_DESC)
        Y_L3                = BreakdownBuilder._nice(L3) + "s" # Plural, as we have multiple level 3 things
        Y_L3_DESC           = BreakdownBuilder._nice(L3_DESC)

        for row in l2l3_df.iterrows(): # nth cycle of loop
            data_i                       = row[1]
            l2_i                         = data_i[L2]
            l2_desc_i                    = BreakdownBuilder._strip(data_i[L2_DESC])
            l3_i                         = data_i[L3]
            l3_desc_i                    = BreakdownBuilder._strip(data_i[L3_DESC])
            
            if BreakdownBuilder.      _is_blank(l2_i): # We are within the same Level 2 as in prior cycle of the loop
                l3_UID_i                 = store.generateUID(acronym=L3[0], parent_UID=l2_UID_n)
                l3_children_n[l3_UID_i]  = l3_i

            elif (type(l2_i)==str and len(l2_i)>0): # We just entered a new Level 2
                l2_UID_n                 = store.generateUID(acronym    = BreakdownBuilder._acronym(L2[0]), 
                                                             parent_UID = link.L1_UID)
                l3_UID_i                 = store.generateUID(acronym    = BreakdownBuilder._acronym(L3[0]), 
                                                             parent_UID = l2_UID_n)
                
                l3_children_n           = {}
                l3_children_n[l3_UID_i] = l3_i
                
                l2_dict_i               = {}
                l2_dict_i[Y_L2_DESC]    = l2_desc_i
                l2_dict_i[Y_L3]         = l3_children_n
                
                all_l2s_dict[l2_UID_n]   = l2_i 
                
                all_l2s_dict[l2_UID_n + '-detail']    = l2_dict_i
            else:
                raise ValueError("Expected a string or a blank, not a '" + str(type(row_exp)) 
                                 + '(row=' + str(row[0]) + " and text is '" + str(row_exp) + "')")
            

        return Y_L2, all_l2s_dict
    
    def _acronym(txt):
        '''
        Returns a string of initials for 'txt', in uppercase
        '''
        stripped_txt = BreakdownBuilder._strip(txt)
        tokens       = stripped_txt.split(' ')
        acronym      = ''.join([token[0].upper() for token in tokens])
        return acronym

In [290]:
next_sheet      = SHEETS[0]
URL             = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + next_sheet

#MANIFESTS_REPO  = os.getcwd() + '/../'+ 'test-manifests-repo'
MANIFESTS_REPO  = os.getcwd() + '/../'+ 'experiments'

In [291]:
L0_SHEET       = "Generated"
l0_url         = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + L0_SHEET
l0_range       = "A1:G30"

In [292]:
store        = UID_Store(start_at_0=True)
L1_ids       = []
LINKS        = []
for idx in range(len(SHEETS)):
    uid      = store.generateUID(acronym='W', parent_UID=None)
    L1_ids.append(uid)
    LINKS.append(L1L2_Link(L1_UID         = 'W.0' + str(idx), \
                           L2_URL         = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + SHEETS[idx], \
                           L2_excel_range = EXPECTATIONS_RANGE))
#L1_ids, LINKS

In [293]:
SHEETS

['Ew0', 'Ew1', 'Ew2', 'Ew3', 'Ew4', 'Ew5', 'Ew6', 'Ew7', 'Ew8', 'Ew9']

In [294]:
# (self, ctx, links, l0_url, l0_excel_range, manifests_repo_dir, uid_store):
builder = BreakdownBuilder(ctx=CTX, links=LINKS, l0_url=l0_url, l0_excel_range=l0_range, \
                           manifests_repo_dir=MANIFESTS_REPO, uid_store=store)
builder.build()

apiVersion: breakdown.a6i.io/v1dev
breakdown:
  defined_by: alejandro.hernandez@finastra.com
  defined_on: 2021-04-27 00:00:00
  streams:
    W.00: Cloud value proposition
    W.00-detail:
      Description: Define value proposition for customers on the Cloud
      Lead: "Product Strategy \u2013 Mike S"
      P metric: TBD
      R metric: TBD
      Support: Andrei
      expectations:
        E.00.00: Analyze segmentation
        E.00.00-detail:
          acceptance-criteria-artifacts:
            A.00.00.00: 'Segmentation tree: consolidated tree of segments (verticals,
              geos, tiers, and breakouts for either) '
            A.00.00.01: Size at each leaf in the segmentation tree
            A.00.00.02: Quantification of how much of that could be cloud
          description: Break Finastra's space into segments and for each measure the
            appetite for cloud
        E.00.01: Analyze "Jobs to be done" / Personas
        E.00.01-detail:
          acceptance-criteria-arti

            A.02.01.00: "Define FusionCloud (SaaS) SOM via FusionOperate: \n*full\
              \ CI/CD automation; \n*multi*tenant environments; \n*multi*tenant ops\
              \ (monitoring, upgrades, \u2026)\n*no customizations; \n*n-1"
            A.02.01.01: "Define Cloud@Scale SOM via FusionOperate: \n*full CI/CD automation;\
              \ \n*single*tenant customer environments; \n*multi*tenant ops (monitoring,\
              \ upgrades, \u2026)\n*no customizations; \n*n-1"
            A.02.01.02: "Define FMS SOM via FusionOperate: \n*full CI/CD automation;\
              \ \n*single*tenant customer environments; \n*single*tenant ops (monitoring,\
              \ upgrades, \u2026)\n*customizations built via FO's software factory\
              \ and FFDC APIs\n*n-3 or better"
            A.02.01.03: Define partner-hosted SOMs by reference to the Finastra-hosted
              SOMs (FusionCloud, Cloud@Scale, FMS)
          description: Define all operating models and how Fusi

      R metric: Number of products with customer self-service capabilities supportability
        embedded made available to customers
      Support: Leigh
      expectations:
        E.08.00: Align with Fusion Operate
        E.08.00-detail:
          acceptance-criteria-artifacts:
            A.08.00.00: .nan
          description: Define how customer support would work for each of the service
            operating models that rely on Fusion Operate (FusionCloud, Cloud@Scale,
            FMS, and partner-hosted)
    W.09: Talent transformation
    W.09-detail:
      Description: 'Define and deliver skills & capabilities required fordelivering
        Cloud faster across organization: Product, GCO, CCO.'
      Lead: Clare
      P metric: "Skills & capabilities plans completed\u200B (by workstream)"
      R metric: Actions from plans in place (training, hires, capabilities addressed)
      Support: Shira, Neeha
      expectations:
        E.09.00: Launch FusionOperate - oriented traini

<h1>Scrap</h1>

In [133]:
L0_SHEET       = "Generated"
l0_url         = EXCEL_FOLDER + '/' + EXPECTATIONS_FILE + ':' + L0_SHEET
l0_range       = "A1:G30"

In [134]:
reader         = xli.ExcelTableReader(url=l0_url, excel_range=l0_range)
l0_df          = reader.read()
l0_df

,UID,Title,Lead,Support,Description,R metric,P metric
0,W0,Cloud value proposition,Product Strategy – Mike S,Andrei,Define value proposition for customers on the ...,TBD,TBD
1,W1,Modernization,Alex,"Leigh, GMs, Ouafa",Deliver the main components of the cloud roadm...,Product [X] to target state [Y] by time [Z]Exa...,# of EPICs delivered in time and budgetExample...
2,W2,Software delivery and production operations,Leigh,"Russ, Bryan, Ouafa, Andrew",FusionOperate: Increase automation through dep...,% Cloud clients compliant with EA operating st...,% automated tests integrated with pipeline
3,W3,N-1 compliance (excl. FMS),Michael M,"GMs, Andrew",Institutionalise a release and sales policy to...,"% of Finastra-hosted clients post Feb 1st, 202...",% of Finastra client contracts signed with N-1...
4,W4,FMS / AMS,Andrew,Neil,Develop and implement a 3-year conversion prog...,FMS accretive SW bookings of [$XM],FMS SW pipeline of [$XM]
5,W5,Cloud sales (excl. FMS),Hannes,Rudy,Develop and implement GTM and deal support acr...,Cloud bookings of [$XM],Cloud pipeline by product by region of [$XM]
6,W6,Pricing,Christian,"Brad B., Bruce B.",Develop and implement pricing approach for Clo...,Pricing approach developed and implemented for...,Delivery of milestones per product
7,W7,IT role in native Cloud solutions,Russ,"Leigh, Bryan","Revamp, rework and modernize network and infra...",[X%] Cloud clients compliant utilizing modern ...,% network deployment automated% products onboa...
8,W8,Customer Support for native Cloud solutions,Michael,Leigh,Implement Cloud customer support model across ...,Number of products with customer self-service ...,Delivery of milestones per product
9,W9,Talent transformation,Clare,"Shira, Neeha",Define and deliver skills & capabilities requi...,"Actions from plans in place (training, hires, ...",Skills & capabilities plans completed​ (by wor...


In [268]:
def _cleanup_UID(txt):
    '''
    If the 'txt' finishes in a number but lacks a dot required for UIDs (e.g., txt="W4" or "W-4" instead of "W.04")
    it will return the correctly formatted txt ("W.04")
    '''
    REGEX = '^[a-zA-Z -\._]*([0-9]+)$'
    m     = _re.match(REGEX, txt.strip(' \n'))
    if m==None or len(m.groups()) != 1:
        raise ValueError("Unable to parse '" + txt + "': expected all digits to be in a contiguous suffix.")
    nb    = int(m.group(1))
    if nb > 100:
        raise ValueError("Invalid UID '" + txt + "':' the numerical value " + str(nb) + " should not exceed 99")
    prefix = txt.split(str(nb))[0]
    prefix = prefix.strip(' \n-_.')
    if nb < 10:
        nb_txt = '0' + str(nb) # From '00' to '09'
    else:
        nb_txt = str(nb) # From '10' to '99'
    return prefix + "." + nb_txt

def _buildLevel1Breakdown(l0_url, l0_excel_range):
    reader                           = xli.ExcelTableReader(url=l0_url, excel_range=l0_excel_range)
    l0_df                            = reader.read()
    columns                          = l0_df.columns

    if len(columns) < 2:
        raise ValueException("Too few columns: need at least two columns, the first two of which should the "\
                            + "UID and the Title for a Level 1 breakout. Happened for url=" + url)
    UID                              = columns[0]
    TITLE                            = columns[1]        
    l0_dict                          = {}
    for row in l0_df.iterrows():
        l1_data                      = row[1]
        l1_uid                       = _cleanup_UID(l1_data[UID])
        l0_dict[l1_uid]              = l1_data[TITLE]
        l0_dict[l1_uid + '-detail']  = {}
        for idx in range(2, len(columns)):
            col                      = columns[idx]
            l1_details_dict          = l0_dict[l1_uid + '-detail']
            l1_details_dict[col]     = l1_data[col]
      
    return l0_dict    

In [269]:
_buildLevel1Breakdown(l0_url=l0_url, l0_excel_range=l0_range)

{'W.00': 'Cloud value proposition',
 'W.00-detail': {'Lead': 'Product Strategy – Mike S',
  'Support': 'Andrei',
  'Description': 'Define value proposition for customers on the Cloud',
  'R metric': 'TBD',
  'P metric': 'TBD'},
 'W.01': 'Modernization',
 'W.01-detail': {'Lead': 'Alex',
  'Support': 'Leigh, GMs, Ouafa',
  'Description': 'Deliver the main components of the cloud roadmap: Microservices, Containerization, Multi-tenancy, UX, Operations / DevOps',
  'R metric': 'Product [X] to target state [Y] by time [Z]Example: Product: Essence to Webapp by end of FY22.',
  'P metric': '# of EPICs delivered in time and budgetExample: By Aug: 20%(20% of what is expected in the 12 months of FY 22 has been done)'},
 'W.02': 'Software delivery and production operations',
 'W.02-detail': {'Lead': 'Leigh',
  'Support': 'Russ, Bryan, Ouafa, Andrew',
  'Description': 'FusionOperate: Increase automation through deployment and DevOps across all cloud propositions (FMS, FusionCloud, etc). Support nat

In [ ]:
reader         = xli.ExcelTableReader(url=l0_url, excel_range=l0_range)
